### Первые тесты библиотек для NER - spacy и stanza

17.10.2021

Что в этом ноутбуке:
- демонстрация работы обеих библиотек по поиску ФИО
- подсчет метрики:
    + процент верно найденных ФИО по отношению к ФИО в ручной разметке (столбец `lib_found`, где lib=библиотека)
- список "лишних" сущностей, которые нашли библиотеки (столбец `lib_more`) - пока просто сравнение "в лоб"
- демонстрация для тех, кто любит покопать вручную :)

In [0]:
import pandas as pd

import stanza
stanza.download('ru')

import spacy
!python3 -m spacy download ru_core_news_sm

import re

In [0]:
# чтение файла. Столбец names - ручная разметка Анны

sample_df = pd.read_csv('10_first_marked.csv')
sample_df

In [0]:
# функции для работы библиотек + переформатирование столбца с ручной разметкой, чтобы сущность = одно слово

def extract_full_name(corpus, lib):
    
    if lib == 'stanza':
        result = stanza_process(corpus)
        
    elif lib == 'spacy':
        result = spacy_process(corpus)
    
    full_name_list = []
    for name in result:
        if re.findall(r'[А-Я]{1}\.', str(name)) != []: # если регулярка что-то нашла
            full_name_list.append(name)
        else:                                     # если регулярка ничего не нашла
            new_name = str(name).split(' ')
            for name_ in new_name:
                full_name_list.append(name_)
            
    return full_name_list
#     return result


def stanza_process(corpus):
    nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner') # этих двух процессоров должно быть достаточно под задачу поиска имен
    doc = nlp(corpus)
    result = [ent.text for ent in doc.ents if ent.type == 'PER']
    return result
    
    
def spacy_process(corpus):
    nlp = spacy.load('ru_core_news_sm')
    doc = nlp(corpus)
    result = [ent for ent in doc.ents if ent.label_ == 'PER']
    return result

In [0]:
# переформатируем разметку names, запускаем модельки

sample_df['names_new'] = 0
sample_df['stanza_res'] = 0
sample_df['spacy_res'] = 0


for index, row in sample_df.iterrows():
    
    # формируем новый столбец разметки
    names_list = str(row['names']).split(', ')
    names_list_new = []
    
    for elem in names_list:
        initials_try = re.findall(r'[А-Я]{1}\.', str(elem))
        if initials_try != []: # если регулярка что-то нашла
            names_list_new.append(elem)
        else:
            new_elem = str(elem).split(' ')
            for name_ in new_elem:
                names_list_new.append(name_)
    
    sample_df.loc[index, 'names_new'] = ', '.join(str(word) for word in names_list_new)
    
    full_name_list1 = extract_full_name(row['text'], 'stanza')
    sample_df.loc[index, 'stanza_res'] = ', '.join(str(word) for word in full_name_list1)
    
    full_name_list2 = extract_full_name(row['text'], 'spacy')
    sample_df.loc[index, 'spacy_res'] = ', '.join(str(word) for word in full_name_list2)

In [0]:
# результат
sample_df

In [0]:
# считаем простенькую метрику как процент совпадающих с разметкой ФИО

sample_df.insert(4, 'stanza_found', '0')
sample_df.insert(5, 'stanza_more', '0')
sample_df['spacy_found'] = '0'
sample_df['spacy_more'] = '0'

for index, row in sample_df.iterrows():
    true = str(row['names_new']).split(', ')
    stanza_found = str(row['stanza_res']).split(', ')
    
    spacy_found = str(row['spacy_res']).split(', ')
    
    sample_df.loc[index, 'stanza_found'] = len(set(stanza_found) & set(true)) / len(set(true))
    sample_df.loc[index, 'spacy_found'] = len(set(spacy_found) & set(true)) / len(set(true))
    
    stanza_more = ', '.join(x for x in set(stanza_found) - set(true))
    sample_df.loc[index, 'stanza_more'] = stanza_more
    
    spacy_more = ', '.join(x for x in set(spacy_found) - set(true))
    sample_df.loc[index, 'spacy_more'] = spacy_more

In [0]:
sample_df

### Можно отметить,

* что в 5 случаях из 10 stanza показывает результат выше 95% + еще один случай с +80%, при этом результаты stanza существенно выше, чем spacy
* в двух случаях метрика spacy выше (примерно в два раза), чем stanza
* еще в двух случаях библиотеки нашли ФИО, при этом в ручной разметке их не было (возможно, подписант?)

In [0]:
# значения в столбце stanza_more - то есть "лишние" сущности, которых не было в ручной разметке
sample_df['stanza_more'].values

In [0]:
#то же самое для библиотеки spacy
sample_df['spacy_more'].values

### Бонус для тех, кто любит ещё посмотреть глазками ^^

In [0]:
# на примере первой строки датафрейма

true = sample_df.loc[0, 'names_new']
stanza_res = sample_df.loc[0, 'stanza_res']
spacy_res = sample_df.loc[0, 'spacy_res']

true_list = [obj.strip() for obj in true.split(', ')]
stanza_list = [obj.strip() for obj in stanza_res.split(', ')]
spacy_list = [obj.strip() for obj in spacy_res.split(', ')]

In [0]:
# нашла stanza, spacy НЕ нашла
set(stanza_list) - set(spacy_list)

In [0]:
# Нашла spacy, stanza НЕ нашла
set(spacy_list) - set(stanza_list)

In [0]:
# есть в разметке, НЕ нашла stanza
set(true_list) - set(stanza_list)

In [0]:
# есть в разметке, НЕ нашла spacy
set(true_list) - set(spacy_list)

In [0]:
len(true_list), len(stanza_list), len(spacy_list)